In [1]:
(require srfi/1)
(require plot/no-gui)

# Constants

In [2]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037)) ; abridged for testing
                          

In [3]:
(define prefound-primes '(1009 1013 1019
                          10007 10009 10037
                          100003 100019 100043
                          1000003 1000033 1000037
                          1000000007 1000000009 1000000021
                          10000000019 10000000033 10000000061
                          100000000003 100000000019 100000000057
                          1000000000039 1000000000061 1000000000063))

In [4]:
(define timestoavg 100000)

In [45]:
(define fermat-times 10)

In [6]:
(define (square x)
  (* x x))

(define (divides? a b)
  (= (remainder b a) 0))

(define (prime? n)
  (= n (smallest-divisor n)))

(define (find-divisor n test-divisor)
  (cond ((> (square test-divisor) n)
         n)
        ((divides? test-divisor n)
         test-divisor)
        (#t (find-divisor
               n
               (advancemethod test-divisor)))))

# 1-21
> Use the smallest-divisor procedure to find the smallest divisor of each of the following numbers: 199, 1999, 19999.

In [7]:
(define (smallest-divisor n)
  (find-divisor n 2)) ; stock method
  ;(find-divisor-integrated n 2)) ; hopefully faster

In [8]:
(define (prime-method x)
  (prime? x))
  ;(fast-prime? x fermat-times))

In [9]:
(define (advancemethod x)
  ;(next x)) ; 1-23 and forward
  (+ x 1)) ; prior to 1-23

In [10]:
(list 199 (smallest-divisor 199) 1999 (smallest-divisor 1999) 19999 (smallest-divisor 19999))

'(199 199 1999 1999 19999 7)

# 1-22
> write a procedure search-for-primes that checks the primality
of consecutive odd integers in a specified range. Use your procedure to find
the three smallest primes larger than 1000; larger than 10,000; larger than
100,000; larger than 1,000,000.

Times were too inconsistent, so I made an averaged version.

In [11]:
(define (avg-timed-prime-test n)
  (newline)
  (display n)
  (avg-start-prime-test n (current-inexact-milliseconds) 0 timestoavg))

(define (avg-start-prime-test n start-time total-time iter)
  (if (prime-method n)
      (let* ((this-time (- (current-inexact-milliseconds)
                          start-time))
            (new-total-time (+ total-time this-time)))
        (if (> iter 0)
          (avg-start-prime-test n (current-inexact-milliseconds) new-total-time (- iter 1))
          (list n (avg-report-prime (* 1.0 (/ new-total-time timestoavg))))))
      #f))
(define (avg-report-prime elapsed-time)
  (display " *** ")
  (display elapsed-time)
  elapsed-time)

(define (avg-search-for-primes minimum goal)
  (define m (if (even? minimum)
                (+ minimum 1)
                (minimum)))
  (define answer (avg-search-for-primes-iter m '() goal))
  (newline)
  answer)
(define (avg-search-for-primes-iter n lst goal)
  (if (= goal 0)
      lst
      (let ((x (avg-timed-prime-test n)))
        (if (not (equal? x #f))
            (avg-search-for-primes-iter (+ n 2) (cons x lst) (- goal 1))
            (avg-search-for-primes-iter (+ n 2) lst goal)))))

In [12]:
(define prime-times-set (append-map (lambda n (avg-search-for-primes (car n) 3)) '(10 100 1000 10000 100000 1000000)))


11 *** 0.0003431689453125
13 *** 0.00024562255859375
15
17 *** 0.00027622802734375

101 *** 0.0005096337890625
103 *** 0.000672861328125
105
107 *** 0.00049347412109375

1001
1003
1005
1007
1009 *** 0.00123931396484375
1011
1013 *** 0.00123373046875
1015
1017
1019 *** 0.00122795654296875

10001
10003
10005
10007 *** 0.00366612548828125
10009 *** 0.00503373291015625
10011
10013
10015
10017
10019
10021
10023
10025
10027
10029
10031
10033
10035
10037 *** 0.0036471923828125

100001
100003 *** 0.011121337890625
100005
100007
100009
100011
100013
100015
100017
100019 *** 0.01129197021484375
100021
100023
100025
100027
100029
100031
100033
100035
100037
100039
100041
100043 *** 0.0111245556640625

1000001
1000003 *** 0.03478732421875
1000005
1000007
1000009
1000011
1000013
1000015
1000017
1000019
1000021
1000023
1000025
1000027
1000029
1000031
1000033 *** 0.0348555908203125
1000035
1000037 *** 0.0350156884765625


In [13]:
prime-times-set

'((17 0.00027622802734375) (13 0.00024562255859375) (11 0.0003431689453125) (107 0.00049347412109375) (103 0.000672861328125) (101 0.0005096337890625) (1019 0.00122795654296875) (1013 0.00123373046875) (1009 0.00123931396484375) (10037 0.0036471923828125) (10009 0.00503373291015625) (10007 0.00366612548828125) (100043 0.0111245556640625) (100019 0.01129197021484375) (100003 0.011121337890625) (1000037 0.0350156884765625) (1000033 0.0348555908203125) (1000003 0.03478732421875))

In [14]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (function (lambda (x) (* (sqrt x) 0.00002))
                           #:color 'blue #:style 'dot
                           #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(11 1000037) #(4894474295696533/73786976294838206464 630786566300673/18014398509481984)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

It takes roughly $\sqrt{10}$ times longer to verify a prime 10 times higher.

# 1.23

define a procedure next that returns 3 if its input is equal
to 2 and otherwise returns its input plus 2. Modify the smallest-divisor
procedure to use `(next test-divisor)` instead of `(+ test-divisor 1)`.

In [15]:
;; We'll be using prefound primes which is larger than the original problem definition,
;;   so let's update this dataset before changing methods
(define prime-times-set (map avg-timed-prime-test prefound-primes))
prime-times-set


1009 *** 0.0012561328125
1013 *** 0.0013040576171875
1019 *** 0.00126231201171875
10007 *** 0.00367068603515625
10009 *** 0.00364358154296875
10037 *** 0.003665185546875
100003 *** 0.01112237548828125
100019 *** 0.01117121826171875
100043 *** 0.01114485595703125
1000003 *** 0.03470440185546875
1000033 *** 0.03478986083984375
1000037 *** 0.0347492431640625
1000000007 *** 1.0998548974609375
1000000009 *** 1.0997917651367188
1000000021 *** 1.100276328125
10000000019 *** 3.478043122558594
10000000033 *** 3.4781062890625
10000000061 *** 3.4937154052734374
100000000003 *** 11.075387043457031
100000000019 *** 11.017280869140626
100000000057 *** 11.070458459472656
1000000000039 *** 35.021130493164065
1000000000061 *** 34.803940615234374
1000000000063 *** 34.809371462402346

'((1009 0.0012561328125) (1013 0.0013040576171875) (1019 0.00126231201171875) (10007 0.00367068603515625) (10009 0.00364358154296875) (10037 0.003665185546875) (100003 0.01112237548828125) (100019 0.01117121826171875) (100043 0.01114485595703125) (1000003 0.03470440185546875) (1000033 0.03478986083984375) (1000037 0.0347492431640625) (1000000007 1.0998548974609375) (1000000009 1.0997917651367188) (1000000021 1.100276328125) (10000000019 3.478043122558594) (10000000033 3.4781062890625) (10000000061 3.4937154052734374) (100000000003 11.075387043457031) (100000000019 11.017280869140626) (100000000057 11.070458459472656) (1000000000039 35.021130493164065) (1000000000061 34.803940615234374) (1000000000063 34.809371462402346))

In [16]:
(define (next n)
  (if (= n 2)
      3
      (+ n 2)))

In [17]:
(define (advancemethod x)
  (next x)) ; 1-23 and forward
  ;(+ x 1)) ; prior to 1-23

In [18]:
;; Using prefound primes now
(define prime-times-next (map avg-timed-prime-test prefound-primes))
prime-times-next


1009 *** 0.00085520263671875
1013 *** 0.0008576513671875
1019 *** 0.0008526171875
10007 *** 0.00224420166015625
10009 *** 0.00224032958984375
10037 *** 0.00223952392578125
100003 *** 0.006641689453125
100019 *** 0.0066384130859375
100043 *** 0.00663258544921875
1000003 *** 0.02043068115234375
1000033 *** 0.02035953369140625
1000037 *** 0.02036950927734375
1000000007 *** 0.6484302197265625
1000000009 *** 0.6487844116210938
1000000021 *** 0.648381337890625
10000000019 *** 2.0363471240234374
10000000033 *** 2.03454099609375
10000000061 *** 2.034465041503906
100000000003 *** 6.345154282226562
100000000019 *** 6.34512431640625
100000000057 *** 6.398720187988281
1000000000039 *** 20.026649997558593
1000000000061 *** 20.05789600830078
1000000000063 *** 20.03680803466797

'((1009 0.00085520263671875) (1013 0.0008576513671875) (1019 0.0008526171875) (10007 0.00224420166015625) (10009 0.00224032958984375) (10037 0.00223952392578125) (100003 0.006641689453125) (100019 0.0066384130859375) (100043 0.00663258544921875) (1000003 0.02043068115234375) (1000033 0.02035953369140625) (1000037 0.02036950927734375) (1000000007 0.6484302197265625) (1000000009 0.6487844116210938) (1000000021 0.648381337890625) (10000000019 2.0363471240234374) (10000000033 2.03454099609375) (10000000061 2.034465041503906) (100000000003 6.345154282226562) (100000000019 6.34512431640625) (100000000057 6.398720187988281) (1000000000039 20.026649997558593) (1000000000061 20.05789600830078) (1000000000063 20.03680803466797))

In [19]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(1009 1000000000063) #(7864005525329265/9223372036854775808 1232196486243025/35184372088832)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

Comparing (1000000 0.0347) for the old algorithm with (1000000 0.02), it also runs at $n * \sqrt{10}$, but takes about 3/5th the time. The (next)
function should allow for skipping about half the iterations necessary, which
is why I'm surprised it doesn't run in 1/2 the time. I wonder if the repeated
conditionals involved slow it down enough to be noticeable. Doesn't seem
likely. I'm going to look at what others say about this.

*Later...*

I was wrong, it *was* the conditional. I'll try inlining the code and see what can be gained.

In [20]:
(define (find-divisor-integrated n test-divisor)
  ; when first run, runs logic for test-divisor = 2, then proceeds to odd looping
  (define (fdi-iter test-divisor)
    (cond ((> (square test-divisor) n)
           n)
          ((divides? test-divisor n)
           test-divisor)
          (#t (fdi-iter
               (+ test-divisor 2)))))
  (if (divides? test-divisor n) ;; Assuming test-divisor is 2
      test-divisor
      (fdi-iter (+ test-divisor 1))))

In [21]:
(define (smallest-divisor n)
  ;(find-divisor n 2)) ; stock method
  (find-divisor-integrated n 2)) ; hopefully faster

In [22]:
(define prime-times-fdi (map avg-timed-prime-test prefound-primes))
prime-times-fdi


1009 *** 0.000671171875
1013 *** 0.00065459228515625
1019 *** 0.00064629150390625
10007 *** 0.00161585693359375
10009 *** 0.00162593994140625
10037 *** 0.0016214013671875
100003 *** 0.004718330078125
100019 *** 0.0047183251953125
100043 *** 0.00472888671875
1000003 *** 0.01449175537109375
1000033 *** 0.0144987060546875
1000037 *** 0.0145105908203125
1000000007 *** 0.45622768310546874
1000000009 *** 0.45622888671875
1000000021 *** 0.4561706103515625
10000000019 *** 1.447540673828125
10000000033 *** 1.447742802734375
10000000061 *** 1.4446926977539063
100000000003 *** 4.582836845703125
100000000019 *** 4.58275587890625
100000000057 *** 4.584263737792969
1000000000039 *** 14.540000808105468
1000000000061 *** 14.528157080078126
1000000000063 *** 14.525368952636718

'((1009 0.000671171875) (1013 0.00065459228515625) (1019 0.00064629150390625) (10007 0.00161585693359375) (10009 0.00162593994140625) (10037 0.0016214013671875) (100003 0.004718330078125) (100019 0.0047183251953125) (100043 0.00472888671875) (1000003 0.01449175537109375) (1000033 0.0144987060546875) (1000037 0.0145105908203125) (1000000007 0.45622768310546874) (1000000009 0.45622888671875) (1000000021 0.4561706103515625) (10000000019 1.447540673828125) (10000000033 1.447742802734375) (10000000061 1.4446926977539063) (100000000003 4.582836845703125) (100000000019 4.58275587890625) (100000000057 4.584263737792969) (1000000000039 14.540000808105468) (1000000000061 14.528157080078126) (1000000000063 14.525368952636718))

In [23]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 (lines prime-times-fdi #:style 'short-dash
                        #:label "time with inlined function"
                        #:color 'blue)
                 (points prime-times-fdi)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

(plot-pict '(prog #<procedure:...e/no-gui/plot2d.rkt:144:8> 400) 400 400 400 0 '() #f #f '#(#(1009 1000000000063) #(5960986984785725/9223372036854775808 1232196486243025/35184372088832)) #<procedure:...lot2d/plot-area.rkt:202:4> #<procedure:...lot2d/plot-area.rkt:713:4> '#(0 0 1))

NOTE: In the .scm version, this speedup was more drastic, running at twice the speed of the previous generation. This one is less drastic and it's inconsistent whether the speedup shows at all per rendering.

# 1.24
Modify the `timed-prime-test procedure` of Exercise 1.22 to use
`fast-prime?` (the Fermat method), and test each of the 12 primes you found in
that exercise. Since the Fermat test has $Θ(\log{n})$ growth, how would you expect
the time to test primes near 1,000,000 to compare with the time needed to
test primes near 1000? Do your data bear this out? Can you explain any
discrepancy you find?

In [59]:
(define (expmod base exp m)
  (cond ((= exp 0) 1)
        ((even? exp)
         (remainder
          (square (expmod base (/ exp 2) m))
          m))
        (else
         (remainder
          (* base (expmod base (- exp 1) m))
          m))))
(define (rnd-range n) ; Racket's (random) doesn't support huge integers
  (inexact->exact (round (+ 1 (* (random) n)))))
(define (fermat-test n)
  (define (try-it a)
    (write "Trying ")
    (write a)
    (= (expmod a n n) a))
  (try-it (rnd-range (- n 1))))
(define (fast-prime? n times)
  (cond ((= times 0) #t)
        ((fermat-test n)
         (fast-prime? n (- times 1)))
        (else #f)))

In [57]:
(list (rnd-range 100)(rnd-range 1000)(rnd-range 10000)(rnd-range 100000))

'(92 396 1929 48483)

In [25]:
(define (prime-method x)
  ;(prime? x))
  (fast-prime? x fermat-times))

In [ ]:
(define prime-times-fermat (map avg-timed-prime-test prefound-primes))
prime-times-fermat

1009"Trying "304"Trying "283"Trying "614"Trying "12"Trying "465"Trying "845"Trying "256"Trying "359"Trying "805"Trying "46"Trying "407"Trying "757"Trying "907"Trying "853"Trying "702"Trying "158"Trying "149"Trying "402"Trying "35"Trying "882"Trying "180"Trying "370"Trying "83"Trying "23"Trying "392"Trying "587"Trying "776"Trying "619"Trying "914"Trying "808"Trying "818"Trying "397"Trying "558"Trying "523"Trying "180"Trying "546"Trying "75"Trying "906"Trying "741"Trying "164"Trying "28"Trying "276"Trying "74"Trying "594"Trying "806"Trying "75"Trying "626"Trying "889"Trying "898"Trying "440"Trying "33"Trying "223"Trying "166"Trying "381"Trying "371"Trying "532"Trying "813"Trying "736"Trying "86"Trying "926"Trying "44"Trying "561"Trying "505"Trying "301"Trying "514"Trying "1006"Trying "195"Trying "178"Trying "73"Trying "493"Trying "76"Trying "534"Trying "510"Trying "199"Trying "241"Trying "731"Trying "488"Trying "540"Trying "332"Trying "106"Trying "849"Trying "828"Trying "629"Trying "61"T

650"Trying "1002"Trying "173"Trying "422"Trying "632"Trying "578"Trying "893"Trying "871"Trying "823"Trying "396"Trying "456"Trying "955"Trying "88"Trying "322"Trying "368"Trying "790"Trying "711"Trying "135"Trying "439"Trying "258"Trying "309"Trying "405"Trying "218"Trying "187"Trying "546"Trying "986"Trying "944"Trying "223"Trying "695"Trying "261"Trying "572"Trying "549"Trying "433"Trying "298"Trying "926"Trying "991"Trying "945"Trying "141"Trying "17"Trying "912"Trying "880"Trying "849"Trying "103"Trying "281"Trying "698"Trying "389"Trying "220"Trying "523"Trying "563"Trying "18"Trying "782"Trying "832"Trying "390"Trying "515"Trying "887"Trying "821"Trying "447"Trying "59"Trying "328"Trying "499"Trying "539"Trying "406"Trying "543"Trying "452"Trying "605"Trying "674"Trying "192"Trying "413"Trying "782"Trying "466"Trying "112"Trying "200"Trying "783"Trying "562"Trying "973"Trying "136"Trying "7"Trying "170"Trying "954"Trying "331"Trying "296"Trying "204"Trying "447"Trying "374"Tryin

"Trying "816"Trying "576"Trying "440"Trying "732"Trying "257"Trying "444"Trying "334"Trying "316"Trying "857"Trying "972"Trying "826"Trying "108"Trying "147"Trying "23"Trying "184"Trying "714"Trying "852"Trying "334"Trying "501"Trying "308"Trying "379"Trying "834"Trying "389"Trying "155"Trying "136"Trying "724"Trying "1004"Trying "93"Trying "730"Trying "707"Trying "755"Trying "954"Trying "137"Trying "956"Trying "338"Trying "838"Trying "1003"Trying "220"Trying "27"Trying "630"Trying "565"Trying "42"Trying "163"Trying "540"Trying "55"Trying "416"Trying "708"Trying "887"Trying "252"Trying "534"Trying "227"Trying "879"Trying "603"Trying "294"Trying "727"Trying "120"Trying "550"Trying "797"Trying "957"Trying "529"Trying "403"Trying "760"Trying "138"Trying "680"Trying "979"Trying "792"Trying "373"Trying "639"Trying "117"Trying "958"Trying "941"Trying "199"Trying "541"Trying "725"Trying "78"Trying "471"Trying "560"Trying "463"Trying "733"Trying "513"Trying "567"Trying "299"Trying "597"Trying 

16"Trying "587"Trying "421"Trying "633"Trying "349"Trying "238"Trying "679"Trying "228"Trying "142"Trying "777"Trying "815"Trying "351"Trying "590"Trying "139"Trying "329"Trying "588"Trying "351"Trying "287"Trying "681"Trying "314"Trying "42"Trying "456"Trying "937"Trying "713"Trying "909"Trying "953"Trying "429"Trying "235"Trying "499"Trying "666"Trying "909"Trying "708"Trying "376"Trying "576"Trying "162"Trying "641"Trying "124"Trying "542"Trying "985"Trying "69"Trying "738"Trying "414"Trying "899"Trying "765"Trying "876"Trying "786"Trying "888"Trying "781"Trying "791"Trying "473"Trying "257"Trying "404"Trying "942"Trying "864"Trying "968"293"Trying "648"Trying "256"Trying "60"Trying "682"Trying "765"Trying "746"Trying "184"Trying "246"Trying "591"Trying "862"Trying "416"Trying "864"Trying "86"Trying "770"Trying "393"Trying "917"Trying "659"Trying "618"Trying "671"Trying "576"Trying "62"Trying "943"Trying "452"Trying "479"Trying "886"Trying "199"Trying "850"Trying "463"Trying "148"Tr

1019"Trying "681"Trying "Trying "466"Trying "17"Trying "686"Trying "913"Trying "650"Trying "706"Trying "78"Trying "206"Trying "696"Trying "641"Trying "673"Trying "27"Trying "552"Trying "876"Trying "749"Trying "45"Trying "542"Trying "567"Trying "816"Trying "329"Trying "84"Trying "486"Trying "225"Trying "834"Trying "317"Trying "126"Trying "474"Trying "148"Trying "885"Trying "476"Trying "850"Trying "516"Trying "42"Trying "115"Trying "1014"Trying "222"Trying "375"Trying "632"Trying "445"Trying "881"Trying "652"Trying "59"Trying "484"Trying "29"Trying "386"Trying "75"Trying "123"Trying "139"Trying "267"Trying "996"Trying "979"Trying "165"Trying "274"Trying "443"Trying "626"Trying "672"Trying "974"Trying "623"Trying "795"Trying "520"Trying "559"Trying "173"Trying "180"Trying "265"Trying "243"Trying "162"Trying "554"Trying "907"Trying "435"Trying "362"Trying "403"Trying "294"Trying "720"Trying "783"Trying "41"Trying "330"Trying "715"Trying "289"Trying "669"Trying "688"Trying "286"Trying "251"

...

Trying "25284"Trying "18476"Trying "5855"Trying "15287"Trying "24638"Trying "15824"Trying "42459"Trying "41271"Trying "88648"Trying "17889"Trying "50970"Trying "47559"Trying "89190"Trying "42529"Trying "76226"Trying "58028"Trying "74807"Trying "87676"Trying "25854"Trying "18404"Trying "1067"Trying "19873"Trying "54696"Trying "57433"Trying "52195"Trying "39810"Trying "41689"Trying "97940"Trying "43926"Trying "11617"Trying "73105"Trying "66088"Trying "13676"Trying "37321"Trying "97632"Trying "58593"Trying "71230"Trying "18919"Trying "2377"Trying "20933"Trying "36369"Trying "276733153"Trying "53915"Trying "22932"Trying "6654"Trying "81060"Trying "87029"Trying "83232"Trying "48574"Trying "6838"Trying "42835"Trying "77752"Trying "58078"Trying "98106"Trying "68683"Trying "19471"Trying "32138"Trying "41710"Trying "46621"Trying "38386"Trying "76622"Trying "63929"Trying "970"Trying "89044"Trying "84815"Trying "20385"Trying "37849"Trying "70088"Trying "14685"Trying "41068"Trying "30606"Trying "6

"65434"Trying "92419"Trying "99328"Trying "16054"Trying "91004"Trying "67314"Trying "21334"Trying "94047"Trying "35332"Trying "34418"Trying "24185"Trying "81457"Trying "23463"Trying "21955"Trying "1405"Trying "59026"Trying "3424"Trying "25539"Trying "16180"Trying "69537"Trying "9575"Trying "98817"Trying "14135"Trying "300"Trying "87334"Trying "50695"Trying "87363"Trying "73962"Trying "91980"Trying "23473"Trying "56541"Trying "95146"Trying "81753"Trying "9886"Trying "76962"Trying "85720"Trying "33112"Trying "72077"Trying "66638"Trying "44993"Trying "12966"Trying "55272"Trying "62196"Trying "79768"Trying "22664"Trying "74244"Trying "64966"Trying "53592"Trying "49353"Trying "85970"Trying "88217"Trying "79779"Trying "80952"Trying "76640"Trying "75638"Trying "27321"Trying "93176"Trying "60756"Trying "95097"Trying "44383"Trying "63239"Trying "96547"Trying "63686"Trying "88750"Trying "93498"Trying "32310"Trying "38385"Trying "49378"Trying "18395"Trying "70961"Trying "40083"Trying "24481"Tryin

"Trying "98494"Trying "20921"Trying "83814"Trying "32578"Trying "80984"Trying "80413"Trying "25114"Trying "64657"Trying "65591"Trying "38329"Trying "50310"Trying "55090"Trying "13719"Trying "72028"Trying "10573"Trying "63518"Trying "57384"Trying "16868"Trying "27400"Trying "51071"Trying "72470"Trying "14691"Trying "14640"Trying "56735"Trying "55679"Trying "28603"Trying "16498"Trying "26543"Trying "97478"Trying "18597"Trying "68057"Trying "52246"Trying "63918"Trying "97212"Trying "8543"Trying "28821"Trying "94455"Trying "96698"Trying "46683"Trying "42403"Trying "87833"Trying "35174"Trying "294"Trying "59616"Trying "55451"Trying "60846"Trying "381"Trying "41467"Trying "19384"Trying "113"Trying "54449"Trying "32800"Trying "79075"Trying "63026"Trying "73587"Trying "76804"Trying "95917"Trying "48733"Trying "18159"Trying "96411"Trying "92183"Trying "2790"Trying "56675"Trying "18861"Trying "56888"Trying "65754"Trying "64982"Trying "32332"Trying "38476"Trying "73848"Trying "91189"Trying "28794

"Trying "95539"Trying "46157"Trying "77183"Trying "67147"Trying "14312"Trying "84041"Trying "6026"Trying "1969"Trying "22982"Trying "82729"Trying "66597"Trying "70857"Trying "69601"Trying "66051"Trying "88437"Trying "96120"Trying "20258"Trying "94036"Trying "54972"Trying "24907"Trying "14616"Trying "32813"Trying "86627"Trying "83972"Trying "85379"Trying "51235"Trying "74627"Trying "90330"Trying "84701"Trying "4038"Trying "99765"Trying "43161"Trying "47404"Trying "51518"Trying "39239"Trying "58120"Trying "21046"Trying "50670"Trying "35700"Trying "63608"Trying "13057"Trying "89412"Trying "34611"Trying "81978"Trying "60488"Trying "84595"Trying "91144"Trying "20626"Trying "23755"Trying "39849"Trying "16608"Trying "79119"Trying "86385"Trying "41043"Trying "5863"Trying "67912"Trying "34245"Trying "2267"Trying "1686"Trying "39512"Trying "41220"Trying "83350"Trying "75548"Trying "1135"Trying "6218"Trying "42344"Trying "28293"Trying "12392"Trying "47590"Trying "43077"Trying "95875"Trying "86881

"24526"Trying "92390"Trying "64010"Trying "80787"Trying "43874"Trying "63057"Trying "20878"Trying "22388"Trying "71633"Trying "7265"Trying "69722"Trying "88155"Trying "20081"Trying "95026"Trying "96637"Trying "13719"Trying "66520"Trying "24091"Trying "57570"Trying "78766"Trying "14259"Trying "45074"Trying "15520"Trying "1286"Trying "96347"Trying "1438"Trying "45202"Trying "62773"Trying "5931"Trying "94317"Trying "14746"Trying "97245"Trying "65490"Trying "61359"Trying "89622"Trying "80242"Trying "77921"Trying "96096"Trying "81877"Trying "31307"Trying "88998"Trying "97755"Trying "2189"Trying "89106"Trying "61468"Trying "75628"Trying "61543"Trying "94010"Trying "55894"Trying "37374"Trying "67059"Trying "6871"Trying "68693"Trying "86044"Trying "52401"Trying "45720"Trying "93842"Trying "11285"Trying "16983"Trying "69868"Trying "81229"Trying "17301"Trying "6966"Trying "99497"Trying "86056"Trying "13277"Trying "3091"Trying "12273"Trying "80748"Trying "62219"Trying "76869"Trying "31341"Trying 

In [46]:
(parameterize ([plot-x-transform  log-transform]
                 [plot-x-ticks      (log-ticks)]
               [plot-y-transform  log-transform]
                 [plot-y-ticks      (log-ticks)])
(plot-pict (list (lines prime-times-set #:style 'short-dash
                        #:label "time to verify prime")
                 (points prime-times-set)
                 (lines prime-times-next #:style 'short-dash
                        #:label "time with next procedure"
                        #:color 'green)
                 (points prime-times-next)
                 (lines prime-times-fdi #:style 'short-dash
                        #:label "time with inlined function"
                        #:color 'blue)
                 (points prime-times-fdi)
                 (lines prime-times-fermat #:style 'short-dash
                        #:label "time for fermat test"
                        #:color 'cyan)
                 ;(function (lambda (x) (* (sqrt x) 0.000015))
                 ;          #:color 'blue #:style 'dot
                 ;          #:label "sqrt(n)")
                 ;(function (lambda (x) (expt 2 n)))
                 )))

lines: contract violation
  expected: a sequence
  given: #f
  in: a part of the or/c of
      an element of
      a part of the or/c of
      the 1st argument of
      (->*
       ((or/c
         natural?
         (sequence/c
          (or/c natural? (sequence/c real?)))))
       (#:alpha
        (>=/c 0)
        #:color
        (or/c
         string?
         symbol?
         (and/c
          exact-integer?
          negative?
          (not/c fixnum?))
         (and/c
          exact-integer?
          positive?
          (not/c fixnum?))
         (and/c fixnum? negative?)
         (and/c fixnum? positive? (not/c index?))
         (and/c index? positive? (not/c byte?))
         g393
         1
         0
         (recursive-contract g422 #:impersonator)
         (list/c real? real? real?))
        #:label
        (or/c string? #f pict?)
        #:style
        (or/c
         (and/c
          exact-integer?
          negative?
          (not/c fixnum?))
         (and/c
          exac